In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.image import imread
import import_ipynb
import utils

In [3]:
train_path, test_path = utils.load_dataset('cell_images')

## Find the average dimension of images

In [4]:
dim1_uninfected, dim2_uninfected = utils.check_image_dimensions(test_path+'uninfected/')

In [5]:
dim1_parasitized, dim2_parasitized = utils.check_image_dimensions(test_path+'parasitized/')

In [6]:
print(np.mean(dim1_uninfected))
print(np.mean(dim2_uninfected))
print(np.mean(dim1_parasitized))
print(np.mean(dim2_parasitized))

130.92538461538462
130.75
134.64307692307693
133.31846153846155


In [7]:
image_shape = (130,130,3)

## Image Manipulation

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [9]:
image_gen = ImageDataGenerator(rotation_range=20, # rotate the image 20 degrees
                               width_shift_range=0.10, # Shift the pic width by a max of 5%
                               height_shift_range=0.10, # Shift the pic height by a max of 5%
                               rescale=1/255, # Rescale the image by normalzing it.
                               shear_range=0.1, # Shear means cutting away part of the image (max 10%)
                               zoom_range=0.1, # Zoom in by 10% max
                               horizontal_flip=True, # Allo horizontal flipping
                               fill_mode='nearest' # Fill in missing pixels with the nearest filled value
                              )

In [10]:
image_gen.flow_from_directory(train_path)

Found 24958 images belonging to 2 classes.


In [11]:
image_gen.flow_from_directory(test_path)

Found 2600 images belonging to 2 classes.


## Create the model

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D

In [13]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())


model.add(Dense(128))
model.add(Activation('relu'))

# Dropouts help reduce overfitting by randomly turning neurons off during training.
# Here we say randomly turn off 50% of neurons.
model.add(Dropout(0.5))

# Last layer, remember its binary so we use sigmoid
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 62, 62, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 29, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 12544)             0

In [15]:
from tensorflow.keras.callbacks import EarlyStopping

In [16]:
early_stop = EarlyStopping(monitor='val_loss',patience=2)

In [17]:
batch_size = 16

In [18]:
train_image_gen = image_gen.flow_from_directory(train_path,
                                               target_size=image_shape[:2],
                                                color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='binary')

Found 24958 images belonging to 2 classes.


In [19]:
test_image_gen = image_gen.flow_from_directory(test_path,
                                               target_size=image_shape[:2],
                                               color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='binary',shuffle=False)

Found 2600 images belonging to 2 classes.


In [20]:
train_image_gen.class_indices

{'parasitized': 0, 'uninfected': 1}

In [21]:
results = model.fit(train_image_gen,epochs=20,
                              validation_data=test_image_gen,
                             callbacks=[early_stop])

Epoch 1/20
1560/1560 [==============================] - 306s 196ms/step - loss: 0.3692 - accuracy: 0.8447 - val_loss: 0.1964 - val_accuracy: 0.9335
Epoch 2/20
1560/1560 [==============================] - 294s 188ms/step - loss: 0.1771 - accuracy: 0.9456 - val_loss: 0.1760 - val_accuracy: 0.9438
Epoch 3/20
1560/1560 [==============================] - 336s 215ms/step - loss: 0.1672 - accuracy: 0.9494 - val_loss: 0.1626 - val_accuracy: 0.9454
Epoch 4/20
1560/1560 [==============================] - 307s 197ms/step - loss: 0.1636 - accuracy: 0.9481 - val_loss: 0.1588 - val_accuracy: 0.9477
Epoch 5/20
1560/1560 [==============================] - 294s 189ms/step - loss: 0.1599 - accuracy: 0.9477 - val_loss: 0.1484 - val_accuracy: 0.9485
Epoch 6/20
1560/1560 [==============================] - 317s 203ms/step - loss: 0.1557 - accuracy: 0.9504 - val_loss: 0.1554 - val_accuracy: 0.9477
Epoch 7/20
1560/1560 [==============================] - 295s 189ms/step - loss: 0.1569 - accuracy: 0.9492 - val_

In [22]:
model.save('malaria_detector.h5')